<a href="https://colab.research.google.com/github/areias/viral-escape/blob/main/COVID_BERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

:
# RoBERTa Transformer Arquitecture Demo on Covid-19 dataset



## 1. Check GPU and RAM specifications

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Feb 25 15:39:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


## 2.  Load Sars-Cov-2 dataset


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# clone repository for language of viral escape paper       
! git clone https://github.com/brianhie/viral-mutation.git

Cloning into 'viral-mutation'...
remote: Enumerating objects: 1114, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 1114 (delta 136), reused 117 (delta 66), pack-reused 890
Receiving objects: 100% (1114/1114), 177.41 MiB | 27.69 MiB/s, done.
Resolving deltas: 100% (744/744), done.


In [5]:
# download data
!wget http://cb.csail.mit.edu/cb/viral-mutation/data.tar.gz


--2022-02-25 14:32:22--  http://cb.csail.mit.edu/cb/viral-mutation/data.tar.gz
Resolving cb.csail.mit.edu (cb.csail.mit.edu)... 128.30.2.148
Connecting to cb.csail.mit.edu (cb.csail.mit.edu)|128.30.2.148|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3060938794 (2.9G) [application/x-gzip]
Saving to: ‘data.tar.gz’

data.tar.gz         100%[===================>]   2.85G  41.7MB/s    in 74s     

2022-02-25 14:33:36 (39.7 MB/s) - ‘data.tar.gz’ saved [3060938794/3060938794]



In [6]:
# unzip data
!tar xvf data.tar.gz

data/
data/escape_results.txt
data/evcouplings/
data/evcouplings/flu_h3_config.yaml
data/evcouplings/flu_h1_config.yaml
data/evcouplings/hiv_bf520_seq.fa
data/evcouplings/hiv_env_seq.fa
data/evcouplings/hiv_env_config.yaml
data/evcouplings/sarscov2_config.yaml
data/evcouplings/flu_h1_seq.fa
data/evcouplings/hiv_bf520_config.yaml
data/evcouplings/sarscov2_seq.fa
data/evcouplings/flu_h3_seq.fa
data/headlines/
data/headlines/abcnews-date-text.csv
data/headlines/headlines.txt
data/hiv/
data/hiv/bg505_regions.txt
data/hiv/fitness_haddox2018/
data/hiv/fitness_haddox2018/BG505-1_prefs.csv
data/hiv/fitness_haddox2018/BG505-3_prefs.csv
data/hiv/fitness_haddox2018/BG505-2_prefs.csv
data/hiv/fitness_haddox2018/BF520-3_prefs.csv
data/hiv/fitness_haddox2018/BF520-2_prefs.csv
data/hiv/fitness_haddox2018/BF520-1_prefs.csv
data/hiv/fitness_haddox2018/BF520_to_HXB2.csv
data/hiv/fitness_haddox2018/BF520_env.fasta
data/hiv/fitness_haddox2018/map_indices.py
data/hiv/fitness_haddox2018/BF520_avgprefs.csv
d

In [7]:
! mkdir drive/MyDrive/Covid-Berta

In [10]:
! mkdir drive/MyDrive/Covid-Berta/data

In [11]:
! cp -r data/cov drive/MyDrive/Covid-Berta/data

In [4]:
! mkdir drive/MyDrive/Covid-Berta/bin


mkdir: cannot create directory ‘drive/MyDrive/Covid-Berta/bin’: File exists


In [9]:
! cp -r viral-mutation/bin/* drive/MyDrive/Covid-Berta/bin

## 3. Data pre-processing
data and code from https://github.com/brianhie/viral-mutation

In [13]:
# install dependencies
! pip install anndata 

     |████████████████████████████████| 91 kB 7.2 MB/s 


In [14]:
! pip install scanpy

     |████████████████████████████████| 2.0 MB 7.4 MB/s 
     |████████████████████████████████| 86 kB 7.3 MB/s 
     |████████████████████████████████| 1.1 MB 58.7 MB/s 
     |████████████████████████████████| 63 kB 2.8 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.2-py3-none-any.whl size=82708 sha256=3b61eaf880d1191f1b1daba9391f15109912e1443aab04a310f2388b53160d41
  Stored in directory: /root/.cache/pip/wheels/84/1b/c6/aaf68a748122632967cef4dffef68224eb16798b6793257d82
  Created wheel for pynndescent: filename=pynndescent-0.5.6-py3-none-any.whl size=53943 sha256=7573e6e6012edf7d960a933d0bd95875416aee442fe7791d7903cac6a6abcdf3
  Stored in directory: /root/.cache/pip/wheels/03/f1/56/f80d72741e400345b5a5b50ec3d929aca581bf45e0225d5c50
  Created wheel for sinfo: filename=sinfo-0.3.4-py3-none-any.whl size=7899 sha256=ed4e13c7eb6c0f2de29344b1b95591dd74237dd9c0c62ae6ce1b413010cdc63f
  Stored in directory: /root/.cache/pip/wheels/68/ca/56/344d532fe53e855ccd6549795d370588ab8123

In [15]:
! pip install bio


     |████████████████████████████████| 271 kB 9.0 MB/s 
     |████████████████████████████████| 2.3 MB 80.1 MB/s 


In [7]:
# add repository mutation to path
import sys
sys.path.append('drive/MyDrive/Covid-Berta/bin')

In [11]:

from utils import Seq, SeqIO

def parse_viprbrc(entry):
    fields = entry.split('|')
    if fields[7] == 'NA':
        date = None
    else:
        date = fields[7].split('/')[0]
        date = dparse(date.replace('_', '-'))

    country = fields[9]
    from locations import country2continent
    if country in country2continent:
        continent = country2continent[country]
    else:
        country = 'NA'
        continent = 'NA'

    from mammals import species2group

    meta = {
        'strain': fields[5],
        'host': fields[8],
        'group': species2group[fields[8]],
        'country': country,
        'continent': continent,
        'dataset': 'viprbrc',
    }
    return meta

def parse_nih(entry):
    fields = entry.split('|')

    country = fields[3]
    from locations import country2continent
    if country in country2continent:
        continent = country2continent[country]
    else:
        country = 'NA'
        continent = 'NA'

    meta = {
        'strain': 'SARS-CoV-2',
        'host': 'human',
        'group': 'human',
        'country': country,
        'continent': continent,
        'dataset': 'nih',
    }
    return meta

def parse_gisaid(entry):
    fields = entry.split('|')

    type_id = fields[1].split('/')[1]

    if type_id in { 'bat', 'canine', 'cat', 'env', 'mink',
                    'pangolin', 'tiger' }:
        host = type_id
        country = 'NA'
        continent = 'NA'
    else:
        host = 'human'
        from locations import country2continent
        if type_id in country2continent:
            country = type_id
            continent = country2continent[country]
        else:
            country = 'NA'
            continent = 'NA'

    from mammals import species2group

    meta = {
        'strain': fields[1],
        'host': host,
        'group': species2group[host].lower(),
        'country': country,
        'continent': continent,
        'dataset': 'gisaid',
    }
    return meta


In [12]:
def process(fnames):
    seqs = {}
    for fname in fnames:
        for record in SeqIO.parse(fname, 'fasta'):
            if len(record.seq) < 1000:
                continue
            if str(record.seq).count('X') > 0:
                continue
            if record.seq not in seqs:
                seqs[record.seq] = []
            if fname == 'drive/MyDrive/Covid-Berta/data/cov/viprbrc_db.fasta':
                meta = parse_viprbrc(record.description)
            elif fname == 'drive/MyDrive/Covid-Berta/data/cov/gisaid.fasta':
                meta = parse_gisaid(record.description)
            else:
                meta = parse_nih(record.description)
            meta['accession'] = record.description
            seqs[record.seq].append(meta)

    with open('drive/MyDrive/Covid-Berta/data/cov/cov_all.fa', 'w') as of:
        for seq in seqs:
            metas = seqs[seq]
            for meta in metas:
                of.write('>{}\n'.format(meta['accession']))
                of.write('{}\n'.format(str(seq)))

    return seqs

In [13]:

def setup(): #(args):
    fnames = [ 'drive/MyDrive/Covid-Berta/data/cov/sars_cov2_seqs.fa',
               'drive/MyDrive/Covid-Berta/data/cov/viprbrc_db.fasta',
               'drive/MyDrive/Covid-Berta/data/cov/gisaid.fasta' ]

    seqs = process(fnames)

    seq_len = max([ len(seq) for seq in seqs ]) + 2
    vocab_size = len(AAs) + 2

    #model = get_model(args, seq_len, vocab_size,
    #                  inference_batch_size=600)

    return seqs #model, seqs


In [14]:
 AAs = [
        'A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H',
        'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W',
        'Y', 'V', 'X', 'Z', 'J', 'U', 'B',
    ]
vocabulary = { aa: idx + 1 for idx, aa in enumerate(sorted(AAs)) }



In [15]:
from dateutil.parser import parse as dparse
seqs = setup()

In [16]:
len(seqs)

4172

In [17]:
seqs[list(seqs.keys())[1]]

[{'accession': 'BCG52843.1 |surface glycoprotein [Severe acute respiratory syndrome coronavirus 2]|Homo sapiens|Japan|Severe acute respiratory syndrome-related coronavirus|',
  'continent': 'Asia (Eastern)',
  'country': 'Japan',
  'dataset': 'nih',
  'group': 'human',
  'host': 'human',
  'strain': 'SARS-CoV-2'},
 {'accession': 'QKS65464.1 |surface glycoprotein [Severe acute respiratory syndrome coronavirus 2]|Homo sapiens|USA|Severe acute respiratory syndrome-related coronavirus|complete',
  'continent': 'America (Northern)',
  'country': 'USA',
  'dataset': 'nih',
  'group': 'human',
  'host': 'human',
  'strain': 'SARS-CoV-2'},
 {'accession': 'QKS65500.1 |surface glycoprotein [Severe acute respiratory syndrome coronavirus 2]|Homo sapiens|USA|Severe acute respiratory syndrome-related coronavirus|complete',
  'continent': 'America (Northern)',
  'country': 'USA',
  'dataset': 'nih',
  'group': 'human',
  'host': 'human',
  'strain': 'SARS-CoV-2'},
 {'accession': 'QKS65512.1 |surface 

In [ ]:
# get sequences subset

In [11]:
def split_seqs(seqs, split_method='random'):
    train_seqs, test_seqs = {}, {}

    tprint('Splitting seqs...')
    for idx, seq in enumerate(seqs):
        if idx % 10 < 2:
            test_seqs[seq] = seqs[seq]
        else:
            train_seqs[seq] = seqs[seq]
    tprint('{} train seqs, {} test seqs.'
           .format(len(train_seqs), len(test_seqs)))

    return train_seqs, test_seqs


In [12]:
import datetime
def tprint(string):
    string = str(string)
    sys.stdout.write(str(datetime.datetime.now()) + ' | ')
    sys.stdout.write(string + '\n')
    sys.stdout.flush()

In [13]:
# using full dataset
train_seqs, test_seqs = split_seqs(seqs)

2022-02-25 15:19:36.661746 | Splitting seqs...
2022-02-25 15:19:36.667739 | 3336 train seqs, 836 test seqs.


In [3]:
import pickle 

In [30]:

with open('drive/MyDrive/Covid-Berta/data/train_seqs.pkl', 'wb') as f:
    pickle.dump(train_seqs, f,protocol=pickle.HIGHEST_PROTOCOL)

In [31]:
with open('drive/MyDrive/Covid-Berta/data/test_seqs.pkl', 'wb') as f:
    pickle.dump(test_seqs, f,protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
with open('drive/MyDrive/Covid-Berta/data/train_seqs.pkl', 'rb') as handle:
   train_seqs = pickle.load(handle)

In [5]:
with open('drive/MyDrive/Covid-Berta/data/test_seqs.pkl', 'rb') as handle:
    test_seqs = pickle.load(handle)

In [24]:
test_seqs[list(test_seqs.keys())[0]]

[{'accession': 'YP_009724390.1 |surface glycoprotein [Severe acute respiratory syndrome coronavirus 2]|Homo sapiens|China|Severe acute respiratory syndrome-related coronavirus|refseq| complete',
  'continent': 'Asia (Eastern)',
  'country': 'China',
  'dataset': 'nih',
  'group': 'human',
  'host': 'human',
  'strain': 'SARS-CoV-2'},
 {'accession': 'BCG52831.1 |surface glycoprotein [Severe acute respiratory syndrome coronavirus 2]|Homo sapiens|Japan|Severe acute respiratory syndrome-related coronavirus|',
  'continent': 'Asia (Eastern)',
  'country': 'Japan',
  'dataset': 'nih',
  'group': 'human',
  'host': 'human',
  'strain': 'SARS-CoV-2'},
 {'accession': 'QKS65476.1 |surface glycoprotein [Severe acute respiratory syndrome coronavirus 2]|Homo sapiens|USA|Severe acute respiratory syndrome-related coronavirus|',
  'continent': 'America (Northern)',
  'country': 'USA',
  'dataset': 'nih',
  'group': 'human',
  'host': 'human',
  'strain': 'SARS-CoV-2'},
 {'accession': 'QKS65488.1 |surf

## 3. Train RoBERTa model

### 3.1. Tokenization with Byte Pair Encoding



> Before amino acid sequences can be interpreted as a language,we must first define what a word is. This is more challenging forproteins than most natural languages because unlike the spacecharacter in languages like English, there is no single character (oramino acid) that is used to divide parts of an amino acid sequenceinto meaningful chunks. In the past, deep learning models haveeither used individual amino acids as input [9,18] or have chosento group every three amino acids as a “word” [4]. However, therehas been recent interest [3] in statistically determining segmentsof amino acids to be used as inputs for downstream machine learn-ing algorithms using an NLP method called byte pair encoding(BPE) [14]. ...The BPE algorithm iteratively merges the most frequent pair of tokens to form a new token [14, 21]. (Nambiar et al., 2020)

> BPE ensures that the most common words are represented in the vocabulary as a single token while the rare words are broken down into two or more subword tokens and this is in agreement with what a subword-based tokenization algorithm does.[source](https://towardsdatascience.com/byte-pair-encoding-subword-based-tokenization-algorithm-77828a70bee0)


In [35]:
! pip install transformers

     |████████████████████████████████| 3.5 MB 9.3 MB/s 
     |████████████████████████████████| 6.8 MB 60.8 MB/s 
     |████████████████████████████████| 596 kB 70.3 MB/s 
     |████████████████████████████████| 67 kB 7.6 MB/s 
     |████████████████████████████████| 895 kB 85.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [36]:
! pip install tokenizers

In [41]:
# make pretraining_data.txt file
for x,seq in enumerate(list(train_seqs.keys())):
    with open("drive/MyDrive/Covid-Berta/pretraining_data.txt", 'a') as filehandle:
        filehandle.write('%s\n' % seq)

In [42]:
# make eval_data.txt file
for x,seq in enumerate(list(test_seqs.keys())):
    with open("drive/MyDrive/Covid-Berta/eval_data.txt", 'a') as filehandle:
        filehandle.write('%s\n' % seq)

In [40]:
!mkdir drive/MyDrive/Covid-Berta/pretraining_data

In [43]:
seq_path = 'drive/MyDrive/Covid-Berta/pretraining_data/'

for x,seq in enumerate(list(train_seqs.keys())):
    with open(seq_path+str(x)+".txt", 'w') as filehandle:
        filehandle.write('%s\n' % seq)

In [44]:
! mkdir drive/MyDrive/Covid-Berta/eval_data

In [45]:
seq_path = 'drive/MyDrive/Covid-Berta/eval_data/'

for x,seq in enumerate(list(test_seqs.keys())):
    with open(seq_path+str(x)+".txt", 'w') as filehandle:
        filehandle.write('%s\n' % seq)

In [46]:
from tokenizers import Tokenizer
from tokenizers.models import BPE

tokenizer = Tokenizer(BPE())

In [47]:
from tokenizers.trainers import BpeTrainer

In [48]:
trainer = BpeTrainer(special_tokens=["<s>", 
                                     "</s>",
                                     "<pad>",
                                "<unk>",
                                "<mask>"],
                     initial_alphabet=list(vocabulary.keys()),
                     vocab_size=10_000)

In [50]:
from pathlib import Path
paths = [str(x) for x in Path("drive/MyDrive/Covid-Berta/pretraining_data").glob("**/*.txt")]


In [51]:
paths[:4]

['drive/MyDrive/Covid-Berta/pretraining_data/0.txt',
 'drive/MyDrive/Covid-Berta/pretraining_data/1.txt',
 'drive/MyDrive/Covid-Berta/pretraining_data/2.txt',
 'drive/MyDrive/Covid-Berta/pretraining_data/3.txt']

In [52]:
tokenizer.train(paths, trainer)

In [54]:
len(str(list(train_seqs.keys())[0]))

1273

In [53]:
str(list(train_seqs.keys())[0])

'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITG

In [55]:
output = tokenizer.encode(str(list(train_seqs.keys())[0]))

In [56]:
output

Encoding(num_tokens=10, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [57]:
print(output.tokens)

['MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQ', 'DSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQID

In [58]:
print(output.ids)

[6169, 6403, 993, 57, 255, 1266, 1563, 1207, 1148, 1146]


In [59]:
tokenizer.token_to_id("<s>")


0

In [60]:
tokenizer.token_to_id("<pad>")


2

In [61]:
from tokenizers.processors import TemplateProcessing

tokenizer.post_processor = TemplateProcessing(single="<s> $A </s>",
                                              special_tokens=[
        ("<s>", tokenizer.token_to_id("<s>")),
        ("</s>", tokenizer.token_to_id("</s>")),
    ]
)

In [62]:
tokenizer.enable_truncation(max_length=512)

In [63]:
tokenizer.enable_padding(pad_token="<pad>")

In [64]:
output = tokenizer.encode(str(list(train_seqs.keys())[100]))

In [65]:
print(output.tokens)

['<s>', 'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRF', 'DNP', 'VLP', 'FN', 'AG', 'VYFA', 'STEKSNIIRGWIFGTT', 'LDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYH', 'KNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWT', 'AGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQGVN', 'CTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQ', 'DSLSSTASALGK

In [66]:
! mkdir drive/MyDrive/Covid-Berta/tokenizer

In [67]:
tokenizer.save("drive/MyDrive/Covid-Berta/tokenizer/tokenizer.json")

In [68]:
tokenizer.model.save("drive/MyDrive/Covid-Berta/tokenizer")

['drive/MyDrive/Covid-Berta/tokenizer/vocab.json',
 'drive/MyDrive/Covid-Berta/tokenizer/merges.txt']

### 3.2.  Train a language model from scratch 

In [6]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [7]:
from transformers import PreTrainedTokenizerFast

fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file="drive/MyDrive/Covid-Berta/tokenizer/tokenizer.json",
                                         pad_token='<pad>',
                                         bos_token='<s>',
                                         eos_token='</s>',
                                         mask_token='<mask>',
                                         unk_token='<unk>',
                                         max_len=512)


In [8]:
# define model config 
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=10_000,
    max_position_embeddings=514, 
    num_attention_heads=12,
    num_hidden_layers=12,
    hidden_size=24,
    type_vocab_size=1
)

In [9]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [10]:
# 93 million parameters 
# with dimension size 20 2 million params
model.num_parameters()


2099632

In [11]:
%%time
from transformers import LineByLineTextDataset

train_dataset = LineByLineTextDataset(
    tokenizer=fast_tokenizer,
    file_path="drive/MyDrive/Covid-Berta/pretraining_data.txt",
    block_size=512,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 4.76 s, sys: 612 ms, total: 5.38 s
Wall time: 2.98 s


In [12]:
eval_dataset = LineByLineTextDataset(
    tokenizer=fast_tokenizer,
    file_path="drive/MyDrive/Covid-Berta/eval_data.txt",
    block_size=512,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [13]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=fast_tokenizer, mlm=True, mlm_probability=0.15
)

In [14]:
#! mkdir drive/MyDrive/Covid-Berta/model

In [15]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="drive/MyDrive/Covid-Berta/model",
    overwrite_output_dir=True,
    num_train_epochs=20,
        do_train = True,
    do_eval = True,
    save_strategy='epoch',
    per_device_train_batch_size=8,
    evaluation_strategy = 'epoch',
    logging_strategy = 'epoch',
    prediction_loss_only=True,
    load_best_model_at_end = True
)

In [16]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset= eval_dataset,
     callbacks= [EarlyStoppingCallback(early_stopping_patience = 1)])

#### Start training

In [17]:

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3336
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 8340


Epoch,Training Loss,Validation Loss
1,9.086800,8.977826
2,8.883800,8.802111
3,8.727100,8.668266
4,8.597200,8.555696
5,8.483700,8.441351
6,8.393500,8.370433
7,8.323700,8.296352
8,8.260800,8.244788
9,8.218800,8.205090
10,8.173000,8.118897


***** Running Evaluation *****
  Num examples = 836
  Batch size = 8
Saving model checkpoint to drive/MyDrive/Covid-Berta/model/checkpoint-417
Configuration saved in drive/MyDrive/Covid-Berta/model/checkpoint-417/config.json
Model weights saved in drive/MyDrive/Covid-Berta/model/checkpoint-417/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 836
  Batch size = 8
Saving model checkpoint to drive/MyDrive/Covid-Berta/model/checkpoint-834
Configuration saved in drive/MyDrive/Covid-Berta/model/checkpoint-834/config.json
Model weights saved in drive/MyDrive/Covid-Berta/model/checkpoint-834/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 836
  Batch size = 8
Saving model checkpoint to drive/MyDrive/Covid-Berta/model/checkpoint-1251
Configuration saved in drive/MyDrive/Covid-Berta/model/checkpoint-1251/config.json
Model weights saved in drive/MyDrive/Covid-Berta/model/checkpoint-1251/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 836
  Batc

TrainOutput(global_step=4587, training_loss=8.480074622830145, metrics={'train_runtime': 480.5111, 'train_samples_per_second': 138.852, 'train_steps_per_second': 17.357, 'total_flos': 127704366537216.0, 'train_loss': 8.480074622830145, 'epoch': 11.0})

In [18]:
trainer.save_model("drive/MyDrive/Covid-Berta")

Saving model checkpoint to drive/MyDrive/Covid-Berta
Configuration saved in drive/MyDrive/Covid-Berta/config.json
Model weights saved in drive/MyDrive/Covid-Berta/pytorch_model.bin
